In [5]:
from pyfaidx import Fasta
sequences = Fasta('uniprot_sprot_varsplic.fasta')

Exception ignored in: <function Faidx.__del__ at 0x7fda307a8310>
Traceback (most recent call last):
  File "/home/max38744/.local/lib/python3.9/site-packages/pyfaidx/__init__.py", line 889, in __del__
    self.__exit__()
  File "/home/max38744/.local/lib/python3.9/site-packages/pyfaidx/__init__.py", line 895, in __exit__
    self.file.close()
AttributeError: 'Faidx' object has no attribute 'file'
Exception ignored in: <function Faidx.__del__ at 0x7fda307a8310>
Traceback (most recent call last):
  File "/home/max38744/.local/lib/python3.9/site-packages/pyfaidx/__init__.py", line 889, in __del__
    self.__exit__()
  File "/home/max38744/.local/lib/python3.9/site-packages/pyfaidx/__init__.py", line 895, in __exit__
    self.file.close()
AttributeError: 'Faidx' object has no attribute 'file'
Exception ignored in: <function Faidx.__del__ at 0x7fda307a8310>
Traceback (most recent call last):
  File "/home/max38744/.local/lib/python3.9/site-packages/pyfaidx/__init__.py", line 889, in __del__

In [9]:
import biotite.database.entrez as entrez
import biotite.sequence as seq
import biotite.sequence.io.fasta as fasta
# Find UIDs for SwissProt/UniProt entries
query =   entrez.SimpleQuery("Avidin", "Protein Name") \
         & entrez.SimpleQuery("Gallus gallus", "Organism") \
         & entrez.SimpleQuery("srcdb_swiss-prot", "Properties")
print(query)

Exception ignored in: <function Faidx.__del__ at 0x7fda307a8310>
Traceback (most recent call last):
  File "/home/max38744/.local/lib/python3.9/site-packages/pyfaidx/__init__.py", line 889, in __del__
    self.__exit__()
  File "/home/max38744/.local/lib/python3.9/site-packages/pyfaidx/__init__.py", line 895, in __exit__
    self.file.close()
AttributeError: 'Faidx' object has no attribute 'file'


((Avidin[Protein Name]) AND ("Gallus gallus"[Organism])) AND (srcdb_swiss-prot[Properties])


In [10]:
uids = entrez.search(query, db_name="protein")

In [12]:
file_name = entrez.fetch_single_file(uids, "avidin.fa", db_name="protein", ret_type="fasta")

In [17]:
fasta_file = fasta.FastaFile()
fasta_file.read(file_name)
print(fasta_file)

>sp|P02701.3|AVID_CHICK RecName: Full=Avidin; Flags: Precursor
MVHATSPLLLLLLLSLALVAPGLSARKCSLTGKWTNDLGSNMTIGAVNSRGEFTGTYITAVTATSNEIKE
SPLHGTQNTINKRTQPTFGFTVNWKFSESTTVFTGQCFIDRNGKEVLKTMWLLRSSVNDIGDDWKATRVG
INIFTRLRTQKE


In [74]:
from SPARQLWrapper import SPARQLWrapper, JSON

def get_seq(uniprot_ids) :
    values = ' '.join(f'<http://purl.uniprot.org/uniprot/{uniprot_id}>' for uniprot_id in uniprot_ids)

    sparql = SPARQLWrapper("https://sparql.uniprot.org/sparql")
    query = f"""
        PREFIX up:<http://purl.uniprot.org/core/>
        SELECT ?protein ?isoform ?aa_sequence
        WHERE {{
            VALUES ?protein {{ {values} }}
            ?protein a up:Protein .
            ?protein up:sequence ?isoform .
            ?isoform rdf:value ?aa_sequence .
        }}
    """
    sparql.setQuery(query)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    pro_list = []
    seq_list = []
    for result in results["results"]["bindings"]:
        pro_list.append(result["protein"]["value"].split("/")[-1])
        seq_list.append(result["aa_sequence"]["value"])
    return pro_list, seq_list


In [75]:
from rdflib import *
from SPARQLWrapper import SPARQLWrapper, JSON

In [78]:
import pandas as pd
import time
from urllib.error import HTTPError
full_protein = np.array(pd.read_csv("pro_list.txt", header = None)[0])


protein_ids = []
sequence = []
for i in range(0, len(full_protein), 99):
    while True:
        try:
            pro_iter, seq_iter = get_seq(full_protein[i:i+99])
            protein_ids.extend(pro_iter)
            sequence.extend(seq_iter)
            break
        except HTTPError as e:
            if e.code == 503:
                time.sleep(2) # 1초 동안 대기
            else:
                raise e



In [111]:
df = pd.DataFrame({"ids" : protein_ids, "seq" : sequence})

In [104]:
df.drop_duplicates(subset=['ids']).sort_index().to_csv("proteins.csv")


In [126]:
df[df['ids'] == "P14416"].drop_duplicates(subset=["seq"]).loc[1].seq

'MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIAVIVFGNVLVCMAVSREKALQTTTNYLIVSLAVADLLVATLVMPWVVYLEVVGEWKFSRIHCDIFVTLDVMMCTASILNLCAISIDRYTAVAMPMLYNTRYSSKRRVTVMISIVWVLSFTISCPLLFGLNNADQNECIIANPAFVVYSSIVSFYVPFIVTLLVYIKIYIVLRRRRKRVNTKRSSRAFRAHLRAPLKEAARRAQELEMEMLSSTSPPERTRYSPIPPSHHQLTLPDPSHHGLHSTPDSPAKPEKNGHAKDHPKIAKIFEIQTMPNGKTRTSLKTMSRRKLSQQKEKKATQMLAIVLGVFIICWLPFFITHILNIHCDCNIPPVLYSAFTWLGYVNSAVNPIIYTTFNIEFRKAFLKILHC'

In [125]:
df[df['ids'] == "P14416"].drop_duplicates(subset=["seq"]).loc[8].seq

'MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIAVIVFGNVLVCMAVSREKALQTTTNYLIVSLAVADLLVATLVMPWVVYLEVVGEWKFSRIHCDIFVTLDVMMCTASILNLCAISIDRYTAVAMPMLYNTRYSSKRRVTVMISIVWVLSFTISCPLLFGLNNADQNECIIANPAFVVYSSIVSFYVPFIVTLLVYIKIYIVLRRRRKRVNTKRSSRAFRAHLRAPLKGNCTHPEDMKLCTVIMKSNGSFPVNRRRVVQEAARRAQELEMEMLSSTSPPERTRYSPIPPSHHQLTLPDPSHHGLHSTPDSPAKPEKNGHAKDHPKIAKIFEIQTMPNGKTRTSLKTMSRRKLSQQKEKKATQMLAIVLGVFIICWLPFFITHILNIHCDCNIPPVLYSAFTWLGYVNSAVNPIIYTTFNIEFRKAFLKILHC'

In [116]:
df.drop_duplicates(subset=['seq'])

,ids,seq
0,P28335,MVNLRNAVHSFLVHLIGLLVWQSDISVSPVAAIVTDIFNTSDGGRF...
1,P14416,MDPLNLSWYDDDLERQNWSRPFNGSDGKADRPHYNYYATLLTLLIA...
3,P24462,MDLIPNLAVETWLLLAVSLILLYLYGTRTHGLFKKLGIPGPTPLPF...
4,P10632,MEPFVVLVLCLSFMLLFSLWRQSCRRRKLPPGPTPLPIIGNMLQID...
5,P22748,MRMLLALLALSAARPSASAESHWCYEVQAESSNYPCLVPVKWGGNC...
...,...,...
23491,P10398,MEPPRGPPANGAEPSRAVGTVKVYLPNKQRTVVTVRDGMSVYDSLD...
23513,P10398,MEPPRGPPANGAEPSRAVGTVKVYLPNKQRTVVTVRDGMSVYDSLD...
23542,P45452,MHPGVLAAFLFLSWTHCRALPLPSGGDEDDLSEEDLQFAERYLRSY...
23550,Q96R05,MPADLSGTWTLLSSDNFEGYMLALGIDFATRKIAKLLKPQKVIEQN...
